import numpy as np
import talib as ta
import csv
import pandas as pd
import matplotlib.pyplot as plt
import skimage
from skimage.transform import resize
import pylab
import imageio
import cv2
from io import BytesIO
import PIL
from talib import MA_Type

ave = []
label=[]
low = []
openn = []
closen = []
image = []

for y in range(2017,2019):
    file = "%d.csv" %y
    print(y)

    df = pd.read_csv(file)
    df = np.array(df)

    sumn = df[0][2]
    count = 1
    newlist = []
    minmute = []

    for i in range(1,len(df)):
        if df[i][1][3] != df[i-1][1][3] or df[i][0] != df[i-1][0] or df[i][1][:2] != df[i-1][1][:2] or i == len(df)-1:
            sumn = sumn / count
            newlist.append(sumn)
            if df[i][0] != df[i-1][0] or df[i][1][:2] != df[i-1][1][:2]:
                minmute.append(1)
            else:
                minmute.append(0)
            sumn = 0
            count = 0

        count += 1
        sumn += df[i][2]

    newlist = np.array(newlist)
    
    for i in range(33,len(minmute)):
        if minmute[i] == 1:
            first = i
            break
            
    for i in range(33,len(minmute)):
        if minmute[i] == 0:
            last = i
    
    fo = first
    sumn = 0
    for i in range(first + 1,last + 1):
        
        if minmute[i] == 1 or i == last:
            
            if i-1 != fo:
                for j in range(fo,i):
                    sumn += newlist[j]
                ave.append(sumn / (i - fo))
                fo = i
    for i in range(1,len(ave)):
        if ave[i] > ave[i - 1]:
            label.append(2)
        elif ave[i] == ave[i - 1]:
            label.append(1)
        else:
            label.append(0)

    a,b,c = ta.MACD(newlist, fastperiod=12, slowperiod=26, signalperiod=9) #5,21,9
    macd = pd.DataFrame(a)
    macdsignal = pd.DataFrame(b)
    macdhist = pd.DataFrame(c)

    a = ta.MOM(newlist, timeperiod=10)
    real = pd.DataFrame(a)

    a, b, c = ta.BBANDS(newlist, matype=MA_Type.T3)
    upper = pd.DataFrame(a)
    middle = pd.DataFrame(b)
    lower = pd.DataFrame(c)

    a = ta.RSI(newlist, timeperiod=14)
    real1 = pd.DataFrame(a)
    fo = first
    lo = 0
    n = 0

    for i in range(first+1,last+1):
        if minmute[i] == 1 or i == last:
            if fo != i-1:
                buffer_ = BytesIO()
                plt.rcParams['figure.figsize'] = (1.0, 1.0)
                plt.axis('off')
                plt.plot(100*macd[fo:i]+50)
                plt.plot(100*macdsignal[fo:i]+50)
                plt.plot(100*macdhist[fo:i]+50)
                plt.plot(real1[fo:i])
                plt.plot(10*real[fo:i]+50)
                plt.plot((upper[fo:i]-55))
                plt.plot((middle[fo:i]-55))
                plt.plot((lower[fo:i]-55))
                plt.savefig(buffer_,format = 'png')
                buffer_.seek(0)
                dataPIL = PIL.Image.open(buffer_)
                fig = np.asarray(dataPIL)
                buffer_.close()
                fig = skimage.transform.resize(fig, (75, 75, 3))
                image.append(fig)
                fo=i
                plt.close()

image = np.array(image[:-1])
image = image.reshape(len(image),3,75,75)
labal = list(zip(openn,closen,low,high))
labal = np.array(labal[1:])

np.save("imaget.npy",image)
np.save("labalt.npy",labal)


testd=np.load("image2017~2018.npy")
testl=np.load("labal2017~2018.npy")
image=np.load("image2000~2001.npy")
testl=np.load("labal2000~2001.npy")
X_train = image
y_train = labal
X_test = testd
y_test = testl

model.compile(loss='categorical_crossentropy',
                  optimizer='adam', 
                  metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data = (X_test , y_test) ,callbacks=[lr_reduce,checkpoint] ,epochs=epochs)
model.save(output_model_file)



---

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.models import Sequential , Model, load_model
from tensorflow.keras.layers import Dense , Activation
from tensorflow.keras.layers import Dropout , GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import SGD , RMSprop , Adadelta , Adam
from tensorflow.keras.layers import Conv2D , BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras import backend as K
# K.set_image_dim_ordering('th')
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
from tensorflow.keras.callbacks import ReduceLROnPlateau , ModelCheckpoint , LearningRateScheduler
from tensorflow.keras.backend import set_image_data_format


In [2]:
set_image_data_format('channels_first')

lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=1, verbose=1)
#create the base pre-trained model
base_model = InceptionV3(weights= 'imagenet', include_top=False , input_shape=(3, 75, 75))
x = base_model.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
model.compile(loss='categorical_crossentropy',optimizer='adam')
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3, 75, 75)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 37, 37)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 37, 37)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 37, 37)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:

batch_size = 16
epochs = 3
X_test = np.load("imaget.npy")
y_test = np.load("1labalt.npy")
y_test = to_categorical(y_test, num_classes = None, dtype = 'float32')
for j in range(epochs):
    for i in range(1,7):
        
        X_train = np.load('image%d.npy' %i)
        y = np.load("1labal%d.npy"%i)
        y_train = to_categorical(y, num_classes = None, dtype = 'float32')
#         model.compile(loss='mean_squared_error',
#                       optimizer='sgd')
        print(y_train)
        if j!=0 or i!=1:
            load_model('03.h5')
        history = model.fit(X_train, y_train, validation_data = (X_test , y_test), batch_size=batch_size)
        model.save('03.h5')
        predict=model.predict(X_train)
        np.save("predict%d"%i,predict)
        del X_train

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]
Train on 13518 samples, validate on 12420 samples
13518/13518 [==============================] - 87s 6ms/sample - loss: 0.7549 - val_loss: 2.1376
[[0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 18602 samples, validate on 12420 samples
18602/18602 [==============================] - 95s 5ms/sample - loss: 0.7477 - val_loss: 1.6778
[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]
Train on 18591 samples, validate on 12420 samples
18591/18591 [==============================] - 96s 5ms/sample - loss: 

In [ ]:
    y = np.load('labal1.npy') 
    print(y.shape)
    predict=model.predict(X_test)
    np.save("predict_test",predict)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
labal=np.load('labalt.npy')
predict=np.load('predict_test.npy')
l=(labal.T)*40+100
p=(predict.T)*40+100
plt.plot(p[3][1000:1030])
plt.plot(l[3][1000:1030])
print(l[0][1000:1030])
print(p[0][1000:1030])
plt.show()

import numpy as np
#for i in range(1,7):
l=np.load('labalt.npy')
l=l.T
#l=(l-100)/40
np.save('labalt.npy',l)


In [ ]:
import numpy as np
labal = np.load('labal1.npy')
print(labal)+


In [ ]:
print(label)